In [8]:
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks
import time

start = time.time()

DEV = False
argvs = sys.argv
argc = len(argvs)

if argc > 1 and (argvs[1] == "--development" or argvs[1] == "-d"):
  DEV = True

if DEV:
  epochs = 2
else:
  epochs = 20

train_data_path = 'data/training'
validation_data_path = 'data/validation'

"""
Parameters
"""
img_width, img_height = 128, 128
batch_size = 32
samples_per_epoch = 1000
validation_steps = 300
nb_filters1 = 32
nb_filters2 = 64
conv1_size = 3
conv2_size = 2
pool_size = 2
classes_num = 3
lr = 0.0004

model = Sequential()
model.add(Convolution2D(nb_filters1, conv1_size, conv1_size, border_mode ="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(nb_filters2, conv2_size, conv2_size, border_mode ="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), dim_ordering='th'))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(classes_num, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=lr),
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

"""
Tensorboard log
"""
log_dir = './tf-log/'
tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)
cbks = [tb_cb]

model.fit_generator(
    train_generator,
    samples_per_epoch=samples_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=cbks,
    validation_steps=validation_steps)

target_dir = './models/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
model.save('./models/model.h5')
model.save_weights('./models/weights.h5')

#Calculate execution time
end = time.time()
dur = end-start

if dur<60:
    print("Execution Time:",dur,"seconds")
elif dur>60 and dur<3600:
    dur=dur/60
    print("Execution Time:",dur,"minutes")
else:
    dur=dur/(60*60)
    print("Execution Time:",dur,"hours")

/home/duicane/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(128, 128,..., padding="same")`
/home/duicane/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:48: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (2, 2), padding="same")`
/home/duicane/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`


Found 5500 images belonging to 3 classes.
Found 2022 images belonging to 3 classes.


/home/duicane/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:95: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=20, validation_data=<keras_pre..., callbacks=[<keras.ca..., validation_steps=300, steps_per_epoch=31)`


Epoch 1/20
31/31 [==============================] - 11s 349ms/step - loss: 2.1602 - acc: 0.4929 - val_loss: 0.9719 - val_acc: 0.5639
Epoch 2/20
31/31 [==============================] - 9s 303ms/step - loss: 0.8614 - acc: 0.6260 - val_loss: 0.8443 - val_acc: 0.6299
Epoch 3/20
31/31 [==============================] - 10s 309ms/step - loss: 0.7592 - acc: 0.6865 - val_loss: 0.6994 - val_acc: 0.7011
Epoch 4/20
31/31 [==============================] - 10s 310ms/step - loss: 0.7342 - acc: 0.6825 - val_loss: 0.6466 - val_acc: 0.7311
Epoch 5/20
31/31 [==============================] - 10s 308ms/step - loss: 0.7172 - acc: 0.7046 - val_loss: 0.6027 - val_acc: 0.7566
Epoch 6/20
31/31 [==============================] - 10s 319ms/step - loss: 0.6914 - acc: 0.7088 - val_loss: 0.6016 - val_acc: 0.7495
Epoch 7/20
31/31 [==============================] - 10s 311ms/step - loss: 0.6420 - acc: 0.7218 - val_loss: 0.6289 - val_acc: 0.7368
Epoch 8/20
31/31 [==============================] - 10s 307ms/step - l

In [9]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, load_model

start = time.time()

#Define Path
model_path = './models/model.h5'
model_weights_path = './models/weights.h5'
test_path = 'data/validation'

#Load the pre-trained models
model = load_model(model_path)
model.load_weights(model_weights_path)

#Define image parameters
img_width, img_height = 128, 128

#Prediction Function
def predict(file):
  x = load_img(file, target_size=(img_width,img_height))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  array = model.predict(x)
  result = array[0]
  print(result)
  answer = np.argmax(result)
  if answer == 1:
    print("Predicted: Abstract")
  elif answer == 0:
    print("Predicted: Landscape")
  elif answer == 2:
    print("Predicted: Portrait")

  return answer

#Walk the directory for every image
for i, ret in enumerate(os.walk(test_path)):
  for i, filename in enumerate(ret[2]):
    if filename.startswith("."):
      continue
    
    print(ret[0] + '/' + filename)
    result = predict(ret[0] + '/' + filename)
    print(" ")

#Calculate execution time
end = time.time()
dur = end-start

if dur<60:
    print("Execution Time:",dur,"seconds")
elif dur>60 and dur<3600:
    dur=dur/60
    print("Execution Time:",dur,"minutes")
else:
    dur=dur/(60*60)
    print("Execution Time:",dur,"hours")

data/validation/abstract/resized_scaled_arshile-gorky_soft-night.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_alice-baber_red-yellow-and-blue-1960.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_mostafa-dashti_untitled-2007(1).jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_sam-francis_untitled-sf74-189-1974.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_bradley-walker-tomlin_number-9-in-praise-of-gertrude-stein-1950-1.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_arshile-gorky_golden-brown-painting-1944.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_joan-snyder_symphony-1970.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_richard-diebenkorn_albuquerque-no-3.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_arthur-pinajian_untitled-landscape-woo

[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_alice-baber_noble-numbers-1965.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_arthur-pinajian_untitled-1960.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_joe-goode_clouds-1973.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_raoul-ubac_composition-56.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_jock-macdonald_legend-of-the-orient-1958.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_sam-francis_untitled-sf27s-1975.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_miriam-schapiro_treasury.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_richard-pousette-dart_circle-by-the-falling-waters-1980.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_blinky-palermo_ohne-titel-mit-rotem-str

[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_joan-mitchell_untitled-1979.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_perle-fine_the-roaring-wind-1958.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_atsuko-tanaka_untitled-1999.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_sam-francis_untitled-from-pasadena-box-1963-1.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_joan-mitchell_untitled-1981.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_raoul-ubac_terre-brulee-1976.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_vasile-dobrian_memories-are-kept-in-the-drawer-of-memory-from-the-archways-series.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_richard-diebenkorn_urbana-no-2-the-archer.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/r

[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_blinky-palermo_komposition-blau-rot-auf-weiss-1965.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_sam-francis_untitled-monotype-with-woodcut.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_paul-jenkins_not_detected_235713.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_paul-reed_17-1964.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_richard-diebenkorn_painting-no-ii.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_paul-jenkins_phenomena-tarot-over.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_atsuko-tanaka_untitled-1961.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_perle-fine_untitled-brown-red-green-white-1950.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_blinky-palermo

[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_betty-parsons_untitled-3.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_ralph-rosenborg_an-italian-landscape-with-trees-1966.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_arthur-pinajian_untitled-landscape-bellport-no-1168-1984.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_atsuko-tanaka_77r-84-1984.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_joan-snyder_woman-child-1972.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_paul-reed_20-1962.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_ralph-rosenborg_landscape-a-country-road-and-wild-flowers-1960.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/abstract/resized_scaled_arthur-pinajian_untitled-landscape-bellport-no-732-1992.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation

...

[0. 0. 1.]
Predicted: Portrait
 
data/validation/portrait/resized_scaled_theo-van-rysselberghe_emile-verhaeren-writing-1915.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_mary-cassatt_sara-in-a-bonnet.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_john-singer-sargent_claude-monet-painting-by-the-edge-of-a-wood-1885.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_pierre-bonnard_madame-claude-anet-1910.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_pierre-auguste-renoir_gabrielle-in-the-garden-at-cagnes-1911.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_augustus-john_the-girl-on-the-cliff-1936.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_james-ensor_old-lady-with-blue-shawl-the-artist-s-grandmother-1881.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_paul-gauguin_clovis-188

[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_paul-cezanne_man-in-a-room-1890.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_karoly-ferenczy_man-sitting-on-a-log-1895.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_giovanni-boldini_madame-marie-louise-herouet.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_paul-gauguin_young-woman-at-the-window-1888.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_pierre-auguste-renoir_woman-reading-1909.jpg
[6.984149e-12 0.000000e+00 1.000000e+00]
Predicted: Portrait
 
data/validation/portrait/resized_scaled_edouard-manet_madame-manet-at-bellevue-1880.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_william-merritt-chase_man-with-bandana-1878.jpg
[0. 0. 1.]
Predicted: Portrait
 
data/validation/portrait/resized_scaled_paul-cezanne_man-with-crossed-arms.jpg
[1. 0. 0.]
Predicted:

[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_camille-pissarro_portrait-of-monsieur-louis-estruc.jpg
[0. 0. 1.]
Predicted: Portrait
 
data/validation/portrait/resized_scaled_pierre-auguste-renoir_head-of-a-young-girl-in-red-1916.jpg
[0. 0. 1.]
Predicted: Portrait
 
data/validation/portrait/resized_scaled_vladimir-dimitrov_unknown-title-30.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_henri-de-toulouse-lautrec_the-streetwalker-also-known-as-casque-d-or-1891.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_william-merritt-chase_contemplation.jpg
[2.698201e-28 0.000000e+00 1.000000e+00]
Predicted: Portrait
 
data/validation/portrait/resized_scaled_david-burliuk_a-girl-1954.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_kees-van-dongen_woman-with-cherries-on-her-hat-1905.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_octav-bancila_ig-ncu-cu-

[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_valentin-serov_portrait-of-alexander-lensky-and-alexander-yuzhin-1908.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_henri-de-toulouse-lautrec_emile-davoust-1889.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_paul-gauguin_tahitian-woman-with-flower-in-her-hair.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_robert-delaunay_man-with-a-tulip-also-known-as-portrait-of-jean-metzinger.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_edgar-degas_rose-caron.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_berthe-morisot_on-a-bench.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_guy-rose_portrait-of-an-algerian-1900.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_gwen-john_girl-holding-a-rose.jpg
[1. 0. 0.]
Predict

[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_pierre-auguste-renoir_gabrielle-reading-1906.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_mary-cassatt_young-girl-with-brown-hair-1886.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_edgar-degas_little-girl-carrying-flowers-in-her-apron.jpg
[0. 0. 1.]
Predicted: Portrait
 
data/validation/portrait/resized_scaled_raoul-dufy_marie-max-1927.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_camille-pissarro_the-little-country-maid-1882.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_pablo-picasso_woman-with-hat-1901.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_matej-sternen_ena-fotke.jpg
[1. 0. 0.]
Predicted: Landscape
 
data/validation/portrait/resized_scaled_nicolae-tonitza_the-man-of-a-new-world-portrait-of-a-writer-and-polititian-gala-galaction-1920.jpg
[1. 0. 0